<a href="https://colab.research.google.com/github/jeskowagner/IGC-ADSWorkshop/blob/main/challenges/2023-05-12/parkinson-walkthrough-python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section 1: (Re)introduction of dataset & preparation [Reminder]

This work-along demonstrates how Parkinson's can be predicted from speech patterns using statistics and machine learning in Python.

It uses this (<https://archive.ics.uci.edu/ml/datasets/parkinsons>) dataset.

The authors have kindly extracted some information from spoken interviews of participants.


Generally, you can hover over the square brackets at the top left of the cell. When a white arrow inside a grey circle appears you can click this to run the cell.

Hover your mouse over the bottom of a cell to see the options for new markdown or code cell pop up.

We will get started right away by loading in everything we will need for this tutorial.

You will notice towards the end of the cell we are defining a function to perform logistic regression on some inputs. If you have any questions about this please ask one of the organisers!

In [ ]:
# Install shap for use later
!pip install shap

# Import libraries
import pandas as pd
import shap
import sklearn
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef, ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_score

# Download the data
!curl -o "parkinsons.csv" https://raw.githubusercontent.com/jeskowagner/IGC-ADSWorkshop/main/challenges/2023-05-12/parkinsons.csv


# Set resolution of plots
mpl.rcParams["figure.dpi"] = 150

In [ ]:
# Load in data and simplify for the purpose of this tutorial by choosing a few features.
data = pd.read_csv("./parkinsons.csv")
data = data[["MDVP:Fo(Hz)",'status', "DFA","spread1", "PPE", "D2"]]

#Create a function to generate and evaluate a model
def Gen_and_Eval_Model(X_train, X_test, y_train, y_test):
  
  # Create an instance of the LogisticRegression class
  model = LogisticRegression(max_iter=1000)

  # Fit the model to the training data
  model.fit(X_train, y_train)

  # Predict probabilities for the test data
  y_prob_pred = model.predict_proba(X_test) * 100

  # Predict labels using a 50% threshold
  y_pred = model.predict(X_test)

  # Format the predictions into a dataframe
  y_prob_pred_df = pd.DataFrame(y_prob_pred, 
                              columns=["% No Parkinson's", "% Parkinson's"], 
                              index=y_test.index)

  y_prob_pred_df.insert(2, "Predicted", y_pred)

  y_prob_pred_df.insert(0, "Actual", y_test)
  y_prob_pred_df.groupby("Actual").head(5)

  ConfusionMatrixDisplay.from_predictions(y_test, y_pred, display_labels=["Healthy", "Parkinson's"])

  # Calculate MCC on test data
  mcc = matthews_corrcoef(y_test, y_pred)
  print("MCC on test data:", mcc)


# Section 2: reminder
Some (many) datasets have missing values in them. We need to find these in order to train our model effectively. We can see an example NaN ("Not a number") in the very first cell!

In [ ]:
# Print the top 5 rows of the data
data.head(5)

We can use a simple script here to give us the total missing values in the dataset. We need to use ".sum()" twice because we are summing over both rows and columns.

In [ ]:
# Display the number of missing values in the whole dataset
print(f"There are {data.isna().sum().sum()} missing values")

We can look at where these missing values appear.

In [ ]:
# Find the columns with missing values in them.
data.isna().sum()

We can see here that all of the missing values are in a single column. In these cases it may be best simply to remove the troublesome column in order to train our model, and hope that the information in that column is not too important.

If the missing values are spread throughout the dataset, we will need to use more advanced techniques to deal with the missingness, which we will cover later.

# Section 3: Train/Test Split
In machine learning, we often want to know how well our model performs on our data.

But if we test a trained model on the same data it was trained on, our results will be biased: after all, the model has already seen the data it is being tested on.

One of the ways to tackle this is to split your data into a training set and a test set.

All decisions about the model are made on the training set, and the test set is exclusively used to evaluate the model in the end.

In other words, we want to pause before starting with our analysis and set aside a portion of our data to test on later.

In practice, this means that we often take, say, 80% of our observations for training, and 20% for testing.

This could look like the following code. Note: you will have to fill something in for the proportion of the split.

In [ ]:
# Split the data into Train/Test

# Remove the "status" column from the features and store it in the target variable

# We will store variables used for prediction in "X" and the status of people (healthy, Parkinson's) in "y"
X = data.drop(['status'], axis=1) # a dataframe of all variables we want to use to predict Parkinson's
y = data['status'] # contains 1s and 0s

# Split the data into training and testing sets
# Here (FILLMEIN) is where you can select the proportion of data used for testing: 0.5 would be half, 0.2 would be 20% etc.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2023, shuffle=True)

# How many rows and columns are in the training and testing data?
print(f"Training Data Shape: {X_train.shape}")
print(f"Training Labels Shape: {y_train.shape}")

print(f"Testing Data Shape: {X_test.shape}")
print(f"Testing Labels Shape: {y_test.shape}")

# Section 4: Imputation

Sometimes when collecting data, it is unfeasible to collect all data for all participants.

**Exercise**: imagine a table with 10 patients, containing data of patients whose blood glucose and blood pressure were measured.

For one patient we do not have the blood glucose level.

Ultimately, we will need a dataset with no missing data to build our model.

Speculate and discuss with your group how could you handle this missing case.

Of course, the best solution would be to collect all data, but sometimes this may not be possible especially if there are patients involved.

Finished the Exercise above? Good! Here are some ways you may have thought of:

1.  Remove the patient from the dataset

2.  Remove the measurement with missing values from the dataset

3.  Calculate the average blood glucose level across the other patients and use that value for the missing patient

4.  More advanced: use a machine learning model to predict the missing value

All of them find use in real-world Science!


We will create two new datasets with two different techniques for handling missingness. In the first one we simply remove the feature with missing values.

In the second one we fill in missing values with the feature's mean value.


Removing the rows with missing values in is an option, but is likely to introduce bias to the dataset. Why might this be?

If the value is missing not at random (MNAR) then by removing it you are allowing the reason it is missing to bias the dataset. This in turn will mean that the model will learn incorrect associations and will not generalise to the true population you are studying. For further explanation please ask us!

In [ ]:
X_train_dropped = X_train.drop(["MDVP:Fo(Hz)"], axis=1)
X_test_dropped = X_test.drop(["MDVP:Fo(Hz)"], axis=1)

X_train_imputed = X_train.fillna(X_train.mean())
X_test_imputed = X_test.fillna(X_test.mean())

We can see which of the two techniques for dealing with missing data is more effective for this dataset.

In [ ]:
Gen_and_Eval_Model(X_train_dropped, X_test_dropped, y_train, y_test)

In [ ]:
Gen_and_Eval_Model(X_train_imputed, X_test_imputed, y_train, y_test)

In this case, the two models perform exactly the same as each other. This means that the feature itself was not important to the prediction. For the sake of argument, we will choose the imputation model to continue with.

**Exercise**: does the result match your expectations? Why or why not?

Brainstorm with your group about what could be done to improve the imputation strategies.

In [ ]:
X_train = X_train_dropped
X_test = X_test_dropped

# Section 5: feature selection

We can see that our predictions aren't perfect yet.

One thing we can try is to cherry-pick some features and remove others.

This process is called feature selection and often improves downstream performance.

You can think of it as removing noise from the data: by removing surplus features, the model can focus on the features that are actually important.

So, how do we select important features?

There are a few different strategies, but to stay within time today we will focus on one: removing correlated features.

Let's first look at correlations across our features:

In [ ]:
corr_matrix = X_train.corrwith(y_train)
print(corr_matrix)

# Combine X_train and y_train into a single dataframe
train_df = pd.concat([X_train, y_train], axis=1)

# Compute the pairwise correlations between all columns
corr_matrix = train_df.corr()

In [ ]:
sns.heatmap(corr_matrix, cmap="coolwarm", vmin=-1, vmax=1)
plt.title("Pairwise Feature and Outcome Correlations")
plt.show()

**Exercise**: remembering that the diagnosis (Parkinson's or healthy) is saved in the "status" column, which features correlate with the diagnosis?

Conversely, which ones seems irrelevant?

Which features have high correlation with each other?

Sometimes, removing features that provide similar information can improve model performance.

The idea behind this is to remove noise from our measurements and focus on features that provide unique information.

Given the correlation plot we observed above, let's see how we could remove features that have high correlation to each other.

In [ ]:
# Drop one of each pair of correlated columns:
X_train = X_train.drop(["PPE"], axis=1)
X_test = X_test.drop(["PPE"], axis=1)

Did this improve our performance now? Let's check!

In [ ]:
Gen_and_Eval_Model(X_train, X_test, y_train, y_test)

**Exercise**: does the result match your expectations? Why or why not?

Brainstorm with your group about what might have happened.

# Section 6: cross-validation

Earlier we used a train/test split to build a model and evaluate its performance on held-out data.

However, what if the split we used was unlucky and we got a bad split?

To make sure we use each data point at least once for training and testing (separately), we can use cross-validation.

Cross-validation is a really useful method because it helps us figure out how well our model will do on new, unseen data, without getting too caught up in the specifics of the data we used to train it.

Plus, it helps us make sure our model isn't just memorizing the training data and isn't actually any good at generalizing to new examples.

Cross-validation splits the dataset into `k` equal-sized folds and trains the model on `k`-1 folds while using the remaining fold as a test set.

This process is repeated `k` times, each time using a different fold as the test set.

The results from each fold are then averaged to obtain an overall estimate of model performance.

This allows us to obtain a more robust estimate of the model's generalization performance and reduce the risk of overfitting to the training data.

In [ ]:
# Perform cross validation
model = LogisticRegression(max_iter=1000)
scores = cross_val_score(model, X_train, y_train, cv=5)
print(scores)

**Exercise**: What does this result mean?

Brainstorm with your group about how to interpret this number.

# Section 8: feature importance / model explainability

If you've got this far, well done!

Something else we can do with the model/data is to analyse feature importance with SHAP values (https://shap.readthedocs.io/en/latest/example_notebooks/overviews/An%20introduction%20to%20explainable%20AI%20with%20Shapley%20values.html)

Which feature seems to  be doing all the heavy lifting?

There are lots of other options for model explainability that we won't cover but have a google!

In [ ]:
model.fit(X_train, y_train)

# Fits the explainer
explainer = shap.Explainer(model.predict, X_test)
# Calculates the SHAP values - It takes some time
shap_values = explainer(X_test)

shap.plots.bar(shap_values)